# Documentation

https://opennmt.net/CTranslate2/quickstart.html  
https://opennmt.net/CTranslate2/guides/transformers.html  

# Model Conversion

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install ctranslate2 sacremoses transformers[torch]

# 1m

In [ ]:
!ct2-transformers-converter --model Helsinki-NLP/opus-mt-en-de --output_dir /tmp/ctranslate_2_en_de

# 2m

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
tokenizer.save_pretrained('/tmp/ctranslate_2_en_de')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


('/tmp/ctranslate_2_en_de/tokenizer_config.json',
 '/tmp/ctranslate_2_en_de/special_tokens_map.json',
 '/tmp/ctranslate_2_en_de/vocab.json',
 '/tmp/ctranslate_2_en_de/source.spm',
 '/tmp/ctranslate_2_en_de/target.spm',
 '/tmp/ctranslate_2_en_de/added_tokens.json')

In [4]:
!ls -lh /tmp/ctranslate_2_en_de

total 287M
-rw-r--r-- 1 root root  223 May 24 21:52 config.json
-rw-r--r-- 1 root root 284M May 24 21:52 model.bin
-rw-r--r-- 1 root root 971K May 24 21:52 shared_vocabulary.json
-rw-r--r-- 1 root root 751K May 24 21:53 source.spm
-rw-r--r-- 1 root root   74 May 24 21:53 special_tokens_map.json
-rw-r--r-- 1 root root 779K May 24 21:53 target.spm
-rw-r--r-- 1 root root  816 May 24 21:53 tokenizer_config.json
-rw-r--r-- 1 root root 1.4M May 24 21:53 vocab.json


# Docker Container Preparation Script

In [5]:
from nltk import download as nltk_download

nltk_download(
    'punkt',
    download_dir='/usr/share/nltk_data'
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Translation Script

In [6]:
from multiprocessing import cpu_count

import ctranslate2
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

In [7]:
translator_model = ctranslate2.Translator(
    '/tmp/ctranslate_2_en_de',
    device='cpu',
    inter_threads=1,
    intra_threads=cpu_count()
)

tokenizer = AutoTokenizer.from_pretrained('/tmp/ctranslate_2_en_de')

In [8]:
def translator(input_text: str) -> str:
    source = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))
    results = translator_model.translate_batch([source])
    target = results[0].hypotheses[0]
    output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(target))

    return output_text

In [9]:
input_text = (
    'London is the capital and largest city of England, and the United Kingdom, with a population of around 8.8 million, ' +
    'and the largest city in Western Europe by metropolitan area, with a population of 14.8 million. ' +
    'It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea and ' +
    'has been a major settlement for nearly two millennia. The City of London, its ancient core and financial centre, ' +
    'was founded by the Romans as Londinium and retains its medieval boundaries. The City of Westminster, ' +
    'to the west of the City of London, has for centuries hosted the national government and parliament. ' +
    'Since the 19th century, the name "London" also refers to the metropolis around this core, ' +
    'historically split among the counties of Middlesex, Essex, Surrey, Kent, and Hertfordshire, ' +
    'which since 1965 has largely comprised Greater London, which is governed by 33 local authorities and ' +
    'the Greater London Authority.'
)

In [10]:
translator(input_text)

# 13s

'London ist die Hauptstadt und größte Stadt Englands, und das Vereinigte Königreich mit einer Bevölkerung von rund 8,8 Millionen und die größte Stadt Westeuropas durch Metropolregion, mit einer Bevölkerung von 14,8 Millionen. Es steht an der Themse im Südosten Englands an der Spitze einer 50-Meilen (80 km) Mündung bis zur Nordsee und ist seit fast zwei Jahrtausenden eine große Siedlung. Die City of London, ihr alter Kern und Finanzzentrum, wurde von den Römern als Londinium gegründet und behält seine mittelalterlichen Grenzen. Die City of Westminster, im Westen der City of London, beherbergt seit Jahrhunderten die nationale Regierung und das Parlament. Seit dem 19. Jahrhundert, der Name "London" bezieht sich auch auf die Metropole um diesen Kern, historisch aufgeteilt zwischen den Grafschaften Middlesex, Essex, Surrey, Kent und Hertfordshire, die seit 1965 weitgehend Greater London, die von 33 lokalen Behörden und der Greater London Authority regiert.'

In [11]:
sentences = sent_tokenize(input_text)

In [12]:
for sentence in sentences:
    print(sentence)
    print(translator(sentence))
    print('')

# 11s

London is the capital and largest city of England, and the United Kingdom, with a population of around 8.8 million, and the largest city in Western Europe by metropolitan area, with a population of 14.8 million.
London ist die Hauptstadt und größte Stadt Englands, und das Vereinigte Königreich mit einer Bevölkerung von etwa 8,8 Millionen und die größte Stadt Westeuropas nach Metropolregionen, mit einer Bevölkerung von 14,8 Millionen.

It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea and has been a major settlement for nearly two millennia.
Sie liegt an der Themse im Südosten Englands an der Spitze einer 80 km langen Mündung bis zur Nordsee und ist seit fast zwei Jahrtausenden eine bedeutende Siedlung.

The City of London, its ancient core and financial centre, was founded by the Romans as Londinium and retains its medieval boundaries.
Die City of London, ihr altes Kern- und Finanzzentrum, wurde von den Römern als Londini